# 9.1 在函数上添加包装器

In [1]:
from functools import wraps
from time import time
def time_this(func):
    '''
    装饰器提供功能：计时器
    :param func: 
    :return: 
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time()
        retsult = func(*args, **kwargs)
        end = time()
        print(func.__name__,  end-start)
        return retsult
    return wrapper


使用装饰器

In [2]:
@time_this
def countdown(n):
    '''
    Counts down
    :param n: 
    :return: 
    '''
    while n > 0:
        n -= 1

In [4]:
countdown(100000)

countdown 0.00399017333984375


In [5]:
from inspect import signature

signature(countdown)

<Signature (n)>

In [6]:
countdown.__name__

'countdown'

In [7]:
countdown.__doc__

'\n    Counts down\n    :param n: \n    :return: \n    '

In [8]:
countdown.__annotations__

{}

解除装饰器

In [10]:
countdown.__wrapped__(1000000)


9.4 带参数的装饰器
9.5 可自定义属性的装饰器
9.6 带可选参数的装饰器

In [40]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)
    
    # def decorate(func):
    log_name = name if name else func.__module__
    log = logging.getLogger(log_name)
    log_msg = message if message else func.__name__
    
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, log_msg)
        return func(*args, **kwargs)
    
    @attach_wrapper(wrapper)
    def set_level(new_level):
        nonlocal level
        level = new_level
        
    @attach_wrapper(wrapper)
    def set_message(mew_msg):
        nonlocal log_msg
        log_msg = mew_msg
    
    @attach_wrapper(wrapper)
    def get_level():
        return level
    
    wrapper.get_message = lambda : log_msg
    
    return wrapper
    
    # return decorate
    

In [41]:
# 使用方法

@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam')
    
logging.basicConfig(level=logging.DEBUG)

# 添加 log message
add.set_message('Add called')
add(12, 23)
# 获取 message
print(add.get_message())
# 获取 level
print(add.get_level())

AttributeError: 'int' object has no attribute '__module__'

In [42]:
# 改变 log level
add.set_level(logging.WARNING)
print(add(2, 3))

# 不带装饰器
@logged
def add1(x, y):
    return x + y

5


In [43]:
@time_this
@logged(logging.DEBUG)
def countdown(n):
    while n > 0:
        n -= 1

AttributeError: 'int' object has no attribute '__module__'

In [44]:
countdown(100000)

DEBUG:__main__:countdown


countdown 0.005014181137084961


In [45]:
add1(12, 12)

DEBUG:__main__:add1


24